 # CSCL 2023 Automated Multi-Dimensional Analysis of Peer Feedback in Middle School Mathematics

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import random
import nltk
import os
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn import tree, metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


random.seed(20)

In [ ]:
# split data for student level cross validation: one student is nested within one fold
group_dict = dict()
groups = np.array([])

for index, row in df.iterrows():
    s_id = row['created_by']
    if s_id not in group_dict:
        group_dict[s_id] = index
    groups = np.append(groups, group_dict[s_id])
    
# Set up the splitter with 5 splits
gkf = GroupKFold(n_splits = 5)

## bag of words

In [ ]:
# bag of words + neural nets
# import data
df = pd.read_csv('Annotations_final.csv')

# stemming
porter = PorterStemmer() # stemming recovers root words from plurals etc
stemmed_texts = []

X_text = df['annotation_text']

for answer in X_text:
    answer = ' '.join(porter.stem(word) for word in answer.split(' '))
    stemmed_texts.append(answer)

X, y = np.array(stemmed_texts), df['relating_to_self'] # CHANGE y HERE

#vect = CountVectorizer(ngram_range=(1,1), max_features=1000, stop_words="english") #only unigram 313 features
vect = CountVectorizer(ngram_range=(2,2), max_features=1000, stop_words="english") #only bigram 728 features

X = vect.fit_transform(X).toarray()


# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
accuracy_scores = np.array([])

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    model = Sequential()
    model.add(Dense(12, input_shape=(738,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc']
             )
    
    num_epochs = 30
    batch_size = 10
    
    model.fit(
        X_train, 
        y_train, 
        epochs=num_epochs, 
        validation_split=0.1,
        shuffle=True, 
        batch_size=batch_size)
    
    predictions = model.predict(X_test)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

## USE

In [ ]:
# loading universal sentence encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [ ]:
# import data
df = pd.read_csv("Annotations_final.csv").fillna(0)

# extract features as X
X = df[['annotation_text','created_by']]

# extract the prediction variable as y
y = df.comment_answer # CHANGE y HERE

# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
pred = pd.DataFrame()

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):

    X_train = X.iloc[train_index].drop(['created_by'], axis=1)
    X_test = X.iloc[test_index].drop(['created_by'], axis=1)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    # train classifier on this round of training group
    training_embeddings = embed(X_train.annotation_text.to_list())
    
    model = Sequential()
    model.add(Dense(12, input_shape=(512,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc'])
    
    num_epochs = 30
    batch_size = 10
    
   
    model.fit(
        training_embeddings, 
        y_train, 
        epochs=num_epochs, 
        validation_split=0.1,
        shuffle=True, 
        batch_size=batch_size)
    
    # test classifier on this round of testing group
    testing_embeddings = embed(X_test.annotation_text.to_list())
    predictions = model.predict(testing_embeddings)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

    
# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))